In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [7]:
data = pd.read_csv('player_stat_.csv')
data = data.drop(['player'], axis=1)

In [5]:
pd.set_option('display.max_columns', 500)
data = pd.read_csv('player_stat_.csv')
data

,player,G,GS,CG,SHO,W,L,SV,HLD,IP,BF,H,HR,SO,BB,R,ER,K/9,BB/9,K%,BB%,AVG,WHIP,BABIP,LOB%,ERA,FIP,LL,PF,WAR
0,Hyun-Jin Ryu,27,27,1,0,9,9,0,0,182.2,734,153,12,210,58,58,54,10.35,2.27,0.286,0.063,0.232,1.090,0.320,0.780,2.66,2.41,0.50,1.0250,3.8
1,Yu Darvish,28,28,10,6,18,6,0,0,232.0,885,156,5,276,36,42,37,10.71,1.40,0.312,0.041,0.190,0.828,0.280,0.817,1.44,1.46,0.75,0.9870,4.7
2,Merrill Kelly,28,28,0,0,12,7,0,0,158.1,674,152,18,161,47,78,72,9.15,2.67,0.239,0.070,0.250,1.260,0.309,0.717,4.09,4.40,0.50,0.9710,2.0
3,Kwang Hyun Kim,31,30,0,0,17,6,0,0,190.1,786,198,13,180,38,64,53,8.51,1.80,0.229,0.048,0.270,1.240,0.338,0.792,2.51,2.77,0.50,0.9710,0.6
4,Josh Lindblom,30,30,0,0,20,3,0,0,194.2,773,165,13,189,29,57,54,8.74,1.34,0.245,0.038,0.226,1.000,0.285,0.787,2.50,2.87,0.50,0.9380,0.8
5,Masahiro Tanaka,28,27,8,2,24,0,1,0,212.0,822,168,6,183,32,35,30,7.77,1.36,0.223,0.039,0.218,0.943,0.278,0.863,1.27,2.35,0.75,1.0010,2.9
6,Kenta Maeda,29,29,5,0,15,8,0,0,206.3,821,168,5,175,41,49,48,7.63,1.79,0.213,0.050,0.222,1.013,0.282,0.798,2.09,2.38,0.75,0.9950,2.9
7,Shohei Ohtani,21,10,0,1,10,4,0,1,140.0,548,89,4,174,45,33,29,11.19,2.89,0.318,0.082,0.184,0.957,0.277,0.799,1.86,2.28,0.75,0.9870,1.0
8,Yusei Kikuchi,23,23,1,0,14,4,0,0,163.7,654,124,16,153,45,59,56,8.41,2.47,0.234,0.069,0.210,1.033,0.256,0.757,3.08,3.52,0.75,1.0450,0.2
9,Yoshihisa Hirano,58,0,0,0,3,7,29,8,57.3,240,57,5,47,16,19,17,7.38,2.51,0.196,0.067,0.266,1.273,0.321,0.821,2.67,3.58,0.75,0.9870,0.3


In [9]:
data_norm = (data - data.min()) / (data.max() - data.min())
data_norm = data_norm.to_numpy()
data_norm

array([[0.24390244, 0.9       , 0.1       , 0.        , 0.31818182,
        1.        , 0.        , 0.        , 0.72978839, 0.78304598,
        0.72560976, 0.64705882, 0.71179039, 0.95555556, 0.71830986,
        0.72307692, 0.80821918, 0.5625    , 0.7480315 , 0.59259259,
        0.55813953, 0.58876404, 0.7804878 , 0.375     , 0.4929078 ,
        0.32312925, 0.        , 0.52951917, 0.82352941],
       [0.26829268, 0.93333333, 1.        , 1.        , 0.72727273,
        0.66666667, 0.        , 0.        , 1.        , 1.        ,
        0.74390244, 0.23529412, 1.        , 0.46666667, 0.49295775,
        0.46153846, 0.89041096, 0.14423077, 0.95275591, 0.18518519,
        0.06976744, 0.        , 0.29268293, 0.5952381 , 0.06028369,
        0.        , 1.        , 0.29823494, 1.        ],
       [0.26829268, 0.93333333, 0.        , 0.        , 0.45454545,
        0.77777778, 0.        , 0.        , 0.59902333, 0.69683908,
        0.7195122 , 1.        , 0.49781659, 0.71111111, 1.        ,
  

In [64]:
training_set_1 = torch.FloatTensor(data_norm[0:8])
test_set_1 = torch.FloatTensor(data_norm[8:14])
# test_set_1 = torch[8:14]
# training_set_1

In [58]:
nb_player = len(training_set_1)
nb_stats = data_norm.shape[1]

In [59]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(29, 15)
        self.fc2 = nn.Linear(15, 5)
        self.fc3 = nn.Linear(5, 15)
        self.fc4 = nn.Linear(15, 29)
        self.activation = nn.Sigmoid()
        
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

In [68]:
nb_epoch = 200 #200
for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.
    for id_player in range(nb_player):
        input = Variable(training_set_1[id_player]).unsqueeze(0)
        target = input.clone()
        #Select only rating related columns to compute loss
        target_ratings = target[:, :nb_stats]
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            output_ratings = output[:, :nb_stats]
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output_ratings, target_ratings)
            mean_corrector = nb_stats/float(torch.sum(target.data > 0) + 1e-10)
#             print(mean_corrector)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))
    
# Testing the SAE
test_loss = 0
s = 0.
for id_player in range(len(test_set_1)):
    input = Variable(test_set_1[id_player]).unsqueeze(0)
    target = Variable(test_set_1[id_player]).unsqueeze(0)
    target_ratings = target[:, :nb_stats]
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        output_ratings = output[:, :nb_stats]
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output_ratings, target_ratings)
        mean_corrector = nb_stats/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

epoch: 1 loss: tensor(0.2309)
epoch: 2 loss: tensor(0.2308)
epoch: 3 loss: tensor(0.2308)
epoch: 4 loss: tensor(0.2309)
epoch: 5 loss: tensor(0.2311)
epoch: 6 loss: tensor(0.2309)
epoch: 7 loss: tensor(0.2306)
epoch: 8 loss: tensor(0.2305)
epoch: 9 loss: tensor(0.2304)
epoch: 10 loss: tensor(0.2305)
epoch: 11 loss: tensor(0.2308)
epoch: 12 loss: tensor(0.2307)
epoch: 13 loss: tensor(0.2304)
epoch: 14 loss: tensor(0.2302)
epoch: 15 loss: tensor(0.2301)
epoch: 16 loss: tensor(0.2300)
epoch: 17 loss: tensor(0.2301)
epoch: 18 loss: tensor(0.2301)
epoch: 19 loss: tensor(0.2298)
epoch: 20 loss: tensor(0.2298)
epoch: 21 loss: tensor(0.2298)
epoch: 22 loss: tensor(0.2297)
epoch: 23 loss: tensor(0.2298)
epoch: 24 loss: tensor(0.2298)
epoch: 25 loss: tensor(0.2296)
epoch: 26 loss: tensor(0.2294)
epoch: 27 loss: tensor(0.2294)
epoch: 28 loss: tensor(0.2295)
epoch: 29 loss: tensor(0.2293)
epoch: 30 loss: tensor(0.2292)
epoch: 31 loss: tensor(0.2291)
epoch: 32 loss: tensor(0.2291)
epoch: 33 loss: t

In [73]:
w = list(sae.parameters())
w

[Parameter containing:
 tensor([[-0.1132, -0.3299, -0.1745, -0.1161, -0.2307, -0.0153,  0.0016,  0.0619,
          -0.1594, -0.1892, -0.2939, -0.2778, -0.0546,  0.1120, -0.2290, -0.1972,
           0.2376,  0.2115,  0.2775,  0.2647, -0.3885, -0.2582, -0.2341,  0.0351,
          -0.1593, -0.1808,  0.2824,  0.0758, -0.0422],
         [-0.1129, -0.3292, -0.1738, -0.1156, -0.2301, -0.0159,  0.0016,  0.0617,
          -0.1592, -0.1890, -0.2933, -0.2773, -0.0548,  0.1109, -0.2286, -0.1970,
           0.2362,  0.2103,  0.2760,  0.2632, -0.3875, -0.2577, -0.2339,  0.0347,
          -0.1591, -0.1804,  0.2815,  0.0752, -0.0425],
         [-0.1131, -0.3297, -0.1742, -0.1158, -0.2303, -0.0156,  0.0016,  0.0619,
          -0.1592, -0.1891, -0.2937, -0.2779, -0.0546,  0.1113, -0.2289, -0.1972,
           0.2371,  0.2109,  0.2770,  0.2640, -0.3883, -0.2582, -0.2343,  0.0350,
          -0.1594, -0.1808,  0.2825,  0.0753, -0.0426],
         [-0.1129, -0.3292, -0.1737, -0.1154, -0.2298, -0.0162,  0.0016

In [75]:
for name, param in sae.named_parameters():
    if param.requires_grad:
        print (name, param.data)

fc1.weight tensor([[-0.1132, -0.3299, -0.1745, -0.1161, -0.2307, -0.0153,  0.0016,  0.0619,
         -0.1594, -0.1892, -0.2939, -0.2778, -0.0546,  0.1120, -0.2290, -0.1972,
          0.2376,  0.2115,  0.2775,  0.2647, -0.3885, -0.2582, -0.2341,  0.0351,
         -0.1593, -0.1808,  0.2824,  0.0758, -0.0422],
        [-0.1129, -0.3292, -0.1738, -0.1156, -0.2301, -0.0159,  0.0016,  0.0617,
         -0.1592, -0.1890, -0.2933, -0.2773, -0.0548,  0.1109, -0.2286, -0.1970,
          0.2362,  0.2103,  0.2760,  0.2632, -0.3875, -0.2577, -0.2339,  0.0347,
         -0.1591, -0.1804,  0.2815,  0.0752, -0.0425],
        [-0.1131, -0.3297, -0.1742, -0.1158, -0.2303, -0.0156,  0.0016,  0.0619,
         -0.1592, -0.1891, -0.2937, -0.2779, -0.0546,  0.1113, -0.2289, -0.1972,
          0.2371,  0.2109,  0.2770,  0.2640, -0.3883, -0.2582, -0.2343,  0.0350,
         -0.1594, -0.1808,  0.2825,  0.0753, -0.0426],
        [-0.1129, -0.3292, -0.1737, -0.1154, -0.2298, -0.0162,  0.0016,  0.0617,
         -0.15

In [37]:
PATH = './model/baseball.pth'
torch.save(sae.state_dict(), PATH)

In [69]:
outputs = sae(test_set_1)
outputs

tensor([[ 0.2151,  0.7205,  0.5957,  0.5070,  0.5714,  0.6317,  0.0393,  0.1625,
          0.7317,  0.7589,  0.6254,  0.3242,  0.6451,  0.5739,  0.4944,  0.4820,
          0.6655,  0.4587,  0.6663,  0.5001,  0.2897,  0.3888,  0.3542,  0.5444,
          0.3012,  0.3383,  1.0160,  0.3224,  0.5615],
        [ 0.1848,  0.6357,  0.5819,  0.4593,  0.5257,  0.6168,  0.0363,  0.1536,
          0.7069,  0.7238,  0.5485,  0.2291,  0.6597,  0.5929,  0.4235,  0.4126,
          0.7502,  0.5022,  0.7671,  0.5569,  0.1806,  0.3139,  0.2829,  0.5437,
          0.2326,  0.2861,  0.9844,  0.3330,  0.5959],
        [ 0.1977,  0.6718,  0.5878,  0.4796,  0.5451,  0.6232,  0.0376,  0.1574,
          0.7174,  0.7387,  0.5812,  0.2695,  0.6535,  0.5848,  0.4536,  0.4421,
          0.7141,  0.4837,  0.7242,  0.5328,  0.2270,  0.3457,  0.3132,  0.5440,
          0.2618,  0.3083,  0.9978,  0.3285,  0.5813],
        [ 0.2284,  0.7577,  0.6018,  0.5278,  0.5914,  0.6382,  0.0406,  0.1664,
          0.7425,  0.7743

In [71]:
test_set_1

tensor([[0.1463, 0.7667, 0.1000, 0.0000, 0.5455, 0.4444, 0.0000, 0.0000, 0.6294,
         0.6681, 0.5488, 0.8824, 0.4629, 0.6667, 0.7324, 0.7538, 0.3653, 0.6587,
         0.3386, 0.7037, 0.3023, 0.4607, 0.0000, 0.2381, 0.6418, 0.7007, 1.0000,
         0.6512, 0.1176],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0455, 0.7778, 1.0000, 1.0000, 0.0521,
         0.0733, 0.1402, 0.2353, 0.0000, 0.0222, 0.1690, 0.1538, 0.1301, 0.6779,
         0.0394, 0.6667, 0.9535, 1.0000, 0.7927, 0.6190, 0.4965, 0.7211, 1.0000,
         0.2982, 0.1373],
        [0.2195, 0.8667, 0.0000, 0.0000, 0.5909, 0.4444, 0.0000, 0.0000, 0.6636,
         0.7414, 0.6280, 0.4118, 0.6157, 1.0000, 0.7465, 0.7385, 0.7169, 1.0000,
         0.5984, 1.0000, 0.4419, 0.7438, 0.6098, 0.2262, 0.5816, 0.5476, 1.0000,
         1.0000, 0.0000],
        [0.0000, 0.5667, 0.2000, 0.1667, 0.1818, 0.7778, 0.0000, 0.0000, 0.3869,
         0.4052, 0.4390, 0.2941, 0.1878, 0.0889, 0.3803, 0.3846, 0.0000, 0.1635,
         0.0000, 0.1667, 0.6977

In [72]:
result = criterion(test_set_1,outputs)
result

tensor(0.1299, grad_fn=<MeanBackward0>)